In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

ZIP_PATH = "/content/drive/MyDrive/FER2013_zip.zip"
EXTRACT_PATH = "/content/fer2013"

os.makedirs(EXTRACT_PATH, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_PATH)

print("Dataset extracted to:", EXTRACT_PATH)


Dataset extracted to: /content/fer2013


In [ ]:
!pip install torch torchvision matplotlib scikit-learn


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)

        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 7)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)

        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)

        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# -------- Data Augmentation (TRAIN ONLY) --------
train_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# -------- No Augmentation (VALIDATION) --------
val_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# -------- Datasets --------
train_data = datasets.ImageFolder(
    root="/content/fer2013/train",
    transform=train_transform
)

val_data = datasets.ImageFolder(
    root="/content/fer2013/test",
    transform=val_transform
)

# -------- DataLoaders --------
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

print("Classes:", train_data.classes)


Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
model = EmotionCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Loss: {running_loss:.3f} | Train Acc: {acc:.2f}%")


Epoch [1/15] | Loss: 513.429 | Train Acc: 56.76%
Epoch [2/15] | Loss: 505.267 | Train Acc: 56.99%
Epoch [3/15] | Loss: 500.328 | Train Acc: 57.50%
Epoch [4/15] | Loss: 499.517 | Train Acc: 57.86%
Epoch [5/15] | Loss: 491.355 | Train Acc: 58.40%
Epoch [6/15] | Loss: 486.997 | Train Acc: 58.86%
Epoch [7/15] | Loss: 482.619 | Train Acc: 59.24%
Epoch [8/15] | Loss: 478.559 | Train Acc: 59.51%
Epoch [9/15] | Loss: 476.062 | Train Acc: 59.71%
Epoch [10/15] | Loss: 472.610 | Train Acc: 60.25%
Epoch [11/15] | Loss: 468.167 | Train Acc: 60.74%
Epoch [12/15] | Loss: 464.448 | Train Acc: 60.63%
Epoch [13/15] | Loss: 462.998 | Train Acc: 60.96%
Epoch [14/15] | Loss: 458.264 | Train Acc: 61.24%
Epoch [15/15] | Loss: 453.473 | Train Acc: 61.60%


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/weights.pth"
torch.save(model.state_dict(), SAVE_PATH)

print("Model saved to:", SAVE_PATH)


Model saved to: /content/drive/MyDrive/weights.pth


In [ ]:
from google.colab import files
files.download(SAVE_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("Train samples:", len(train_data))
print("Val samples:", len(val_data))



Train samples: 28709
Val samples: 7178


In [ ]:
model.eval()

val_correct = 0
val_total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

val_acc = 100 * val_correct / val_total
print(f"Validation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 61.30%
